In [20]:
# Import Packages
import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime

In [21]:
# Read in shark data
eat = pd.read_excel("SharkData.xlsx", sheet_name = 0)
targets = pd.read_excel("SharkData.xlsx", sheet_name = 1)
drops = pd.read_excel("SharkData.xlsx", sheet_name = 2)
other_factors = pd.read_excel("SharkData.xlsx", sheet_name = 3)

# Data Cleaning

In [22]:
# Clean eating dataset
eat = eat.iloc[1:,:-1]
eat.rename(columns = {"Unnamed: 14":'Etc. Comments', "Pieces Eaten": "Date", 'Total: ' : 'Total'}, inplace = True)

#get rid of null values at end of dataset
eat = eat.head(582)

# Change date to just date format, not datetime
eat['Date'] = pd.to_datetime(eat['Date']).dt.date



eat.reset_index(drop=True,inplace = True)
eat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total,Etc. Comments
0,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,NaN
1,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,NaN
3,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,NaN
4,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,NaN


In [23]:
# Clean Drops Dataset
drops = drops.iloc[1:,:-1]
drops['Drops'] = pd.to_datetime(drops['Drops']).dt.date
drops.rename(columns = {"Drops" : "Date", "Total: ":"Total"}, inplace = True)
drops.reset_index(drop=True,inplace = True)
drops.tail()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total
577,2020-09-22,0.0,0.0,1.0,4.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,10.0
578,2020-09-24,0.0,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,7.0
579,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0
580,2020-09-27,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,3.0
581,2020-09-29,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,4.0


In [24]:
# Clean Targets Dataset
targets = targets.iloc[1:,:-1]
targets.rename(columns = {"Unnamed: 14":'Etc. Comments', "Targets": "Date", "Total: ": "Total"}, inplace = True)

#get rid of null values at end of dataset
targets = targets.head(582)

# Change date to just date format, not datetime
targets['Date'] = pd.to_datetime(targets['Date']).dt.date

targets.reset_index(drop=True,inplace = True)
targets.tail()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total
577,2020-09-22,6.0,1.0,4.0,6.0,5.0,4.0,7.0,2.0,1.0,5.0,4.0,1.0,46.0
578,2020-09-24,8.0,18.0,2.0,9.0,10.0,10.0,11.0,5.0,6.0,3.0,7.0,9.0,98.0
579,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,3.0,5.0,8.0,24.0
580,2020-09-27,10.0,19.0,9.0,10.0,8.0,16.0,3.0,NaN,NaN,NaN,NaN,NaN,75.0
581,2020-09-29,15.0,5.0,6.0,6.0,4.0,4.0,7.0,4.0,1.0,4.0,4.0,1.0,61.0


# Other Factors Data Cleaning and Dummy Variable Creation for Foods


In [25]:
dummy_factors = other_factors[['Saury', 'Blue Runner',
                'Squid', 'Mackerel', 'Herring', 'Sardine',
                'Mazuri Vitamins', 'Garlic', 'Salmon', 'Bonito', 'Bluefish', 'Mahi',
                    'Goggle Eye', 'Humbolt Squid']]
other_dummies = other_factors[['BT/SB Location (1-6)','GR Location (1-6)']]
other_dummies.rename(columns= {"BT/SB Location (1-6)": "BT_SB_Location", "GR Location (1-6)": "GR_Location"}, inplace = True)
for i in dummy_factors:
    dummy_factors[i] = dummy_factors[i].fillna(0)
dummy_factors.rename(columns = {"Blue Runner": "Blue_Runner", "Mazuri Vitamins": "Mazuri_Vitamins", 
                               "Goggle Eye": "Goggle_Eye", "Humbolt Squid": "Humbolt_Squid"}, inplace = True)

/home/stats/.local/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Combine Datasets to get all factors to perform regression on

In [26]:
# Add seasons variables
def season_of_date(date):
    date = pd.to_datetime(date)
    year = str(date.year)
    seasons = {'spring': pd.date_range(start=year+'/03/21', end=year+'/06/20'),
               'summer': pd.date_range(start=year+'/06/21', end=year+'/09/22'),
               'fall': pd.date_range(start=year+'/09/23', end=year+'/12/20')}
    if date in seasons['spring']:
        return 'spring'
    if date in seasons['summer']:
        return 'summer'
    if date in seasons['fall']:
        return 'fall'
    else:
        return 'winter'

In [27]:
eat_summer = eat[eat.Date.map(season_of_date)=="summer"]
eat_summer['summer'] = 1
eat_summer['fall'] = 0
eat_summer['spring'] = 0
eat_fall = eat[eat.Date.map(season_of_date)=="fall"]
eat_fall['summer'] = 0
eat_fall['fall'] = 1
eat_fall['spring'] = 0
eat_winter = eat[eat.Date.map(season_of_date)=="winter"]
eat_winter['summer'] = 0
eat_winter['fall'] = 0
eat_winter['spring'] = 0
eat_spring = eat[eat.Date.map(season_of_date)=="spring"]
eat_spring['summer'] = 0
eat_spring['fall'] = 0
eat_spring['spring'] = 1


/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [28]:
eat_all_seasons = pd.concat([eat_summer,eat_fall,eat_winter,eat_spring])
eat_all_seasons_sort = eat_all_seasons.sort_values(by=["Date"])
eat_season_vars = eat_all_seasons_sort[['summer', 'fall', 'spring']]

In [29]:
drops_summer = drops[eat.Date.map(season_of_date)=="summer"]
drops_summer['summer'] = 1
drops_summer['fall'] = 0
drops_summer['spring'] = 0
drops_fall = drops[eat.Date.map(season_of_date)=="fall"]
drops_fall['summer'] = 0
drops_fall['fall'] = 1
drops_fall['spring'] = 0
drops_winter = drops[eat.Date.map(season_of_date)=="winter"]
drops_winter['summer'] = 0
drops_winter['fall'] = 0
drops_winter['spring'] = 0
drops_spring = drops[eat.Date.map(season_of_date)=="spring"]
drops_spring['summer'] = 0
drops_spring['fall'] = 0
drops_spring['spring'] = 1

/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [30]:
drops_all_seasons = pd.concat([drops_summer,drops_fall,drops_winter,drops_spring])
drops_all_seasons_sort = drops_all_seasons.sort_values(by=["Date"])
drops_season_vars = drops_all_seasons_sort[['summer', 'fall', 'spring']]

In [31]:
targets_summer = targets[eat.Date.map(season_of_date)=="summer"]
targets_summer['summer'] = 1
targets_summer['fall'] = 0
targets_summer['spring'] = 0
targets_fall = targets[eat.Date.map(season_of_date)=="fall"]
targets_fall['summer'] = 0
targets_fall['fall'] = 1
targets_fall['spring'] = 0
targets_winter = targets[eat.Date.map(season_of_date)=="winter"]
targets_winter['summer'] = 0
targets_winter['fall'] = 0
targets_winter['spring'] = 0
targets_spring = targets[eat.Date.map(season_of_date)=="spring"]
targets_spring['summer'] = 0
targets_spring['fall'] = 0
targets_spring['spring'] = 1

/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [32]:
targets_all_seasons = pd.concat([targets_summer,targets_fall,targets_winter,targets_spring])
targets_all_seasons_sort = targets_all_seasons.sort_values(by=["Date"])
targets_season_vars = targets_all_seasons_sort[['summer', 'fall', 'spring']]

In [33]:
# combine eat data
eat_concat1 = pd.concat([eat,dummy_factors,other_dummies, eat_season_vars], axis = 1)
eat_concat = pd.concat([eat_concat1, other_factors["Temperature"]], axis = 1)
eat_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,Bluefish,Mahi,Goggle_Eye,Humbolt_Squid,BT_SB_Location,GR_Location,summer,fall,spring,Temperature
0,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
1,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,...,0.0,0.0,0.0,0.0,NaN,3.0,0,1,0,73.6
3,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,NaN,0,1,0,73.6
4,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.4


In [34]:
# Add eat variable for Covid shut down
start_date = pd.to_datetime('03-14-2020')
end_date = pd.to_datetime('05-09-2020')
conditions = [ (eat_concat['Date'] >= start_date) & (eat_concat['Date'] <= end_date),
              (eat_concat['Date'] < start_date) | (eat_concat['Date'] > end_date)]
values = [1, 0]
eat_concat["covid"] = np.select(conditions, values)

In [35]:
# Add eat variable for light training
light_start = pd.to_datetime('02-15-2018')
conditions_light = [ (eat_concat['Date'] >= light_start), (eat_concat['Date'] < light_start)]
values_light = [1, 0]
eat_concat["light_training"] = np.select(conditions_light, values_light)

In [36]:
# combine drops data
drops_concat1 = pd.concat([drops,dummy_factors,other_dummies, drops_season_vars], axis = 1)
drops_concat = pd.concat([drops_concat1, other_factors["Temperature"]], axis = 1)
drops_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,Bluefish,Mahi,Goggle_Eye,Humbolt_Squid,BT_SB_Location,GR_Location,summer,fall,spring,Temperature
0,2017-12-05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
1,2017-12-07,0.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,0.0,0.0,0.0,0.0,NaN,3.0,0,1,0,73.6
3,2017-12-10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,NaN,0,1,0,73.6
4,2017-12-12,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.4


In [37]:
# Add drop variable for Covid shut down
start_date = pd.to_datetime('03-14-2020')
end_date = pd.to_datetime('05-09-2020')
conditions = [ (drops_concat['Date'] >= start_date) & (drops_concat['Date'] <= end_date),
              (drops_concat['Date'] < start_date) | (drops_concat['Date'] > end_date)]
values = [1, 0]
drops_concat["covid"] = np.select(conditions, values)

In [38]:
# Add drop variable for light training
light_start = pd.to_datetime('02-15-2018')
conditions_light = [ (drops_concat['Date'] >= light_start), (drops_concat['Date'] < light_start)]
values_light = [1, 0]
drops_concat["light_training"] = np.select(conditions_light, values_light)

In [39]:
# combine Target data
targets_concat1 = pd.concat([targets,dummy_factors,other_dummies, targets_season_vars], axis = 1)
targets_concat = pd.concat([targets_concat1, other_factors["Temperature"]], axis = 1)
targets_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,Bluefish,Mahi,Goggle_Eye,Humbolt_Squid,BT_SB_Location,GR_Location,summer,fall,spring,Temperature
0,2017-12-05,3.0,0.0,4.0,3.0,0.0,2.0,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
1,2017-12-07,4.0,5.0,2.0,2.0,5.0,3.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,...,0.0,0.0,0.0,0.0,NaN,3.0,0,1,0,73.6
3,2017-12-10,4.0,7.0,0.0,2.0,2.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,NaN,0,1,0,73.6
4,2017-12-12,3.0,1.0,3.0,2.0,4.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.4


In [40]:
# Add target variable for Covid shut down
start_date = pd.to_datetime('03-14-2020')
end_date = pd.to_datetime('05-09-2020')
conditions = [ (targets_concat['Date'] >= start_date) & (targets_concat['Date'] <= end_date),
              (targets_concat['Date'] < start_date) | (targets_concat['Date'] > end_date)]
values = [1, 0]
targets_concat["covid"] = np.select(conditions, values)

In [41]:
# Add target variable for light training
light_start = pd.to_datetime('02-15-2018')
conditions_light = [ (targets_concat['Date'] >= light_start), (targets_concat['Date'] < light_start)]
values_light = [1, 0]
targets_concat["light_training"] = np.select(conditions_light, values_light)

In [44]:
# Add group feed variable to eat, drop, and target datasets
GroupFeed = []
for row in drops.index:
    if drops.iloc[row,:].isna().sum() > 0:
        GroupFeed.append(0)
    else:
        GroupFeed.append(1)
GroupFeed

eat_concat["GroupFeed"] = GroupFeed
drops_concat["GroupFeed"] = GroupFeed
targets_concat["GroupFeed"] = GroupFeed
targets_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,Humbolt_Squid,BT_SB_Location,GR_Location,summer,fall,spring,Temperature,covid,light_training,GroupFeed
0,2017-12-05,3.0,0.0,4.0,3.0,0.0,2.0,2.0,1.0,1.0,...,0.0,1.0,3.0,0,1,0,73.2,0,0,1
1,2017-12-07,4.0,5.0,2.0,2.0,5.0,3.0,5.0,0.0,0.0,...,0.0,1.0,3.0,0,1,0,73.2,0,0,1
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,...,0.0,NaN,3.0,0,1,0,73.6,0,0,0
3,2017-12-10,4.0,7.0,0.0,2.0,2.0,0.0,0.0,NaN,NaN,...,0.0,1.0,NaN,0,1,0,73.6,0,0,0
4,2017-12-12,3.0,1.0,3.0,2.0,4.0,1.0,1.0,1.0,1.0,...,0.0,1.0,3.0,0,1,0,73.4,0,0,1


# Subset Creation

In [45]:

# All SS Subset
All_SS = eat_concat[['Ross', 'Chandler']]

# All BT Subset
All_BT = eat_concat[['BT1', 'BT2', 'BT3', 'BT4', 'BT5']]

# All GR Subset
All_GR = eat_concat[['GR1', 'GR2', 'GR3', 'GR4', 'GR5']]

# Male Subset
Male = eat_concat[["BT1","BT5","GR1","Ross","Chandler"]]

# Female Subset
female = eat_concat[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]]

# Append all subsets to original datasets
# eat dataset
eat_concat["All_GR"] = eat['GR1'] + eat['GR2'] + eat['GR3'] + eat['GR4'] + eat['GR5']
eat_concat["All_BT"] = eat['BT1'] + eat['BT2'] + eat['BT3'] + eat['BT4'] + eat['BT5']
eat_concat["All_SS"] = eat['Ross'] + eat['Chandler']
eat_concat["male"] = eat[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_concat["female"] = eat[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

# drop dataset
drops_concat["All_GR"] = drops['GR1'] + drops['GR2'] + drops['GR3'] + drops['GR4'] + drops['GR5']
drops_concat["All_BT"] = drops['BT1'] + drops['BT2'] + drops['BT3'] + drops['BT4'] + drops['BT5']
drops_concat["All_SS"] = drops['Ross'] + drops['Chandler']
drops_concat["male"] = drops[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
drops_concat["female"] = drops[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

# target dataset
targets_concat["All_GR"] = targets['GR1'] + targets['GR2'] + targets['GR3'] + targets['GR4'] + targets['GR5']
targets_concat["All_BT"] = targets['BT1'] + targets['BT2'] + targets['BT3'] + targets['BT4'] + targets['BT5']
targets_concat["All_SS"] = targets['Ross'] + targets['Chandler']
targets_concat["male"] = targets[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
targets_concat["female"] = targets[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)


# Create Tables for Poisson Regression Time Series For Pieces Eaten For All Sharks

In [46]:
# create additional time variables for eating
eat_concat['Date'] = pd.to_datetime(eat_concat['Date'])
eat_concat['Day_of_week'] = pd.to_datetime(eat_concat['Date']).dt.dayofweek
eat_concat['Month'] = pd.to_datetime(eat_concat['Date']).dt.month
eat_concat['Day'] = pd.to_datetime(eat_concat['Date']).dt.day
eat_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,light_training,GroupFeed,All_GR,All_BT,All_SS,male,female,Day_of_week,Month,Day
0,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,...,0,1,6.0,22.0,5.0,12.0,21.0,1,12,5
1,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0.0,2.0,1.0,1.0,2.0,3,12,7
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,...,0,0,6.0,NaN,NaN,2.0,4.0,5,12,9
3,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,...,0,0,NaN,8.0,6.0,7.0,7.0,6,12,10
4,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,...,0,1,7.0,16.0,4.0,9.0,18.0,1,12,12


In [47]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concat)) < 0.8
eat_train = eat_concat[mask]
eat_test = eat_concat[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=445
Testing data set length=137


In [48]:
# setup Poisson regression expression
expr = """Total ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + 
summer + fall + spring + GroupFeed"""

In [49]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [50]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Total   No. Observations:                  216
Model:                            GLM   Df Residuals:                      189
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -706.64
Date:                Sun, 06 Dec 2020   Deviance:                       320.99
Time:                        18:14:57   Pearson chi2:                     318.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.4700      1.327     

Saury, Blue Runner, Mackerel, Herring, Bluefish, Light training, summer, and fall were all significant predictors for the number of pieces eaten for all sharks

# Poisson Regression for Number of Drops for All Sharks

In [51]:
# create additional time variables for dropping
drops_concat['Date'] = pd.to_datetime(drops_concat['Date'])
drops_concat['Day_of_week'] = pd.to_datetime(drops_concat['Date']).dt.dayofweek
drops_concat['Month'] = pd.to_datetime(drops_concat['Date']).dt.month
drops_concat['Day'] = pd.to_datetime(drops_concat['Date']).dt.day
drops_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,light_training,GroupFeed,All_GR,All_BT,All_SS,male,female,Day_of_week,Month,Day
0,2017-12-05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,1,2.0,1.0,0.0,0.0,3.0,1,12,5
1,2017-12-07,0.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,...,0,1,2.0,7.0,1.0,4.0,6.0,3,12,7
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,0,0,1.0,NaN,NaN,0.0,1.0,5,12,9
3,2017-12-10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,...,0,0,NaN,1.0,0.0,1.0,0.0,6,12,10
4,2017-12-12,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0,1,1.0,2.0,0.0,0.0,3.0,1,12,12


In [52]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concat)) < 0.8
drops_train = drops_concat[mask]
drops_test = drops_concat[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=477
Testing data set length=105


In [53]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [54]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Total   No. Observations:                  241
Model:                            GLM   Df Residuals:                      214
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -566.64
Date:                Sun, 06 Dec 2020   Deviance:                       378.73
Time:                        18:15:32   Pearson chi2:                     380.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           9.2448      2.932     

Day of week, Day of Month, Temperature, Blue Runner, Bluefish, covid, and light training were all significant predictors for the number of drops for all sharks

# Poisson Regression for Number of Targets for All Sharks

In [55]:
# create additional time variables for Targeting
targets_concat['Date'] = pd.to_datetime(targets_concat['Date'])
targets_concat['Day_of_week'] = pd.to_datetime(targets_concat['Date']).dt.dayofweek
targets_concat['Month'] = pd.to_datetime(targets_concat['Date']).dt.month
targets_concat['Day'] = pd.to_datetime(targets_concat['Date']).dt.day
targets_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,light_training,GroupFeed,All_GR,All_BT,All_SS,male,female,Day_of_week,Month,Day
0,2017-12-05,3.0,0.0,4.0,3.0,0.0,2.0,2.0,1.0,1.0,...,0,1,8.0,11.0,3.0,10.0,12.0,1,12,5
1,2017-12-07,4.0,5.0,2.0,2.0,5.0,3.0,5.0,0.0,0.0,...,0,1,4.0,17.0,9.0,16.0,14.0,3,12,7
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,...,0,0,8.0,NaN,NaN,1.0,7.0,5,12,9
3,2017-12-10,4.0,7.0,0.0,2.0,2.0,0.0,0.0,NaN,NaN,...,0,0,NaN,4.0,11.0,11.0,4.0,6,12,10
4,2017-12-12,3.0,1.0,3.0,2.0,4.0,1.0,1.0,1.0,1.0,...,0,1,11.0,11.0,4.0,9.0,17.0,1,12,12


In [56]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concat)) < 0.8
targets_train = targets_concat[mask]
targets_test = targets_concat[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=467
Testing data set length=115


In [57]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [58]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Total   No. Observations:                  238
Model:                            GLM   Df Residuals:                      211
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1356.6
Date:                Sun, 06 Dec 2020   Deviance:                       1283.6
Time:                        18:15:52   Pearson chi2:                 1.29e+03
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.8722      0.733     

Month, Temperature, Mackerel, Herring, Sardine, Mazuri Vitamins, garlic, salmon, bonito, goggle eye, humbolt squid, light training, spring, and group feeding were all significant predictors for number of targets for all sharks

# Poisson Regression Across Different Species of Sharks

# Sandbar Sharks

In [59]:
expr = """All_SS ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer +
fall + spring + GroupFeed"""

In [60]:
eat_concatSS = eat_concat[eat_concat['All_SS'].notna()]

In [61]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concatSS)) < 0.8
eat_train = eat_concatSS[mask]
eat_test = eat_concatSS[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=354
Testing data set length=85


Eating:

In [62]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [63]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_SS   No. Observations:                  242
Model:                            GLM   Df Residuals:                      215
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -602.55
Date:                Sun, 06 Dec 2020   Deviance:                       425.41
Time:                        18:16:30   Pearson chi2:                     375.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.9391      2.686     

Temperature, Bonito, Bluefish, BT_SB_Location,covid, light training, summer,spring, and Group feeding were all significant predictors for the number of fish eaten for Sandbars

Dropping:

In [64]:
drops_concatSS = drops_concat[drops_concat['All_SS'].notna()]

In [65]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concatSS)) < 0.8
drops_train = drops_concatSS[mask]
drops_test = drops_concatSS[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=354
Testing data set length=84


In [66]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [67]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_SS   No. Observations:                  233
Model:                            GLM   Df Residuals:                      206
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -174.67
Date:                Sun, 06 Dec 2020   Deviance:                       209.56
Time:                        18:16:40   Pearson chi2:                     258.
No. Iterations:                    20                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.4547   2.92e+04  -4.

there were no significant predictors for the number of drops for the Sandbar sharks

Targets:

In [68]:
targets_concatSS = targets_concat[targets_concat['All_SS'].notna()]

In [69]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concatSS)) < 0.8
targets_train = targets_concatSS[mask]
targets_test = targets_concatSS[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=347
Testing data set length=90


In [70]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [71]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_SS   No. Observations:                  237
Model:                            GLM   Df Residuals:                      210
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -966.62
Date:                Sun, 06 Dec 2020   Deviance:                       886.06
Time:                        18:16:55   Pearson chi2:                     864.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.1790      1.724     

Temperature, Saury, Sardine, Mazuri Vitamins, Garlic, Salmon, Bluefish, Goggle Eye, BT_SB_Location, light training, and group feeding were all significant predictors for number of targets for sandbar sharks

# Blacktip Sharks

Eating:

In [72]:
expr = """All_BT ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer +
fall + spring + GroupFeed"""

In [73]:
eat_concatBT = eat_concat[eat_concat['All_BT'].notna()]

In [74]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concatBT)) < 0.8
eat_train = eat_concatBT[mask]
eat_test = eat_concatBT[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=344
Testing data set length=92


In [75]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [76]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_BT   No. Observations:                  234
Model:                            GLM   Df Residuals:                      207
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -699.17
Date:                Sun, 06 Dec 2020   Deviance:                       370.34
Time:                        18:17:21   Pearson chi2:                     369.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.0970      1.660     

Day of Week, Temperature, Blue Runner, Mackerel, Bonito, BT_SB_Location, light training, spring, and Group feeding were all significant predictors for pieces eaten for Black Tips

Dropping:

In [77]:
drops_concatBT = drops_concat[drops_concat['All_BT'].notna()]

In [78]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concatBT)) < 0.8
drops_train = drops_concatBT[mask]
drops_test = drops_concatBT[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=354
Testing data set length=82


In [79]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [80]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_BT   No. Observations:                  234
Model:                            GLM   Df Residuals:                      208
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -489.46
Date:                Sun, 06 Dec 2020   Deviance:                       320.79
Time:                        18:17:30   Pearson chi2:                     320.
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.9826      1.574     

Day of week, Day of month, Blue runner, Herring, Garlic, covid, and light training were all significant predictors for the number of drops for Black Tips.

Targeting:

In [81]:
targets_concatBT = targets_concat[targets_concat['All_BT'].notna()]

In [82]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concatBT)) < 0.8
targets_train = targets_concatBT[mask]
targets_test = targets_concatBT[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=344
Testing data set length=93


In [83]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [84]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_BT   No. Observations:                  231
Model:                            GLM   Df Residuals:                      204
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1092.6
Date:                Sun, 06 Dec 2020   Deviance:                       949.63
Time:                        18:17:40   Pearson chi2:                     969.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.3131      1.069     

Month, Day of month, Temperature, Saury, Sardine, Garlic, Salmon, Bonito, Mahi, BT_SB Location, light training, summer, fall, spring, and group feeding were all significant predictors for the number of targets for Black Tips

# Gray Reef Sharks

Eating:

In [85]:
expr = """All_GR ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer +
fall + spring + GroupFeed"""

In [86]:
eat_concatGR = eat_concat[eat_concat['All_GR'].notna()]

In [87]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concatGR)) < 0.8
eat_train = eat_concatGR[mask]
eat_test = eat_concatGR[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=345
Testing data set length=92


In [88]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [89]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_GR   No. Observations:                  235
Model:                            GLM   Df Residuals:                      209
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -627.10
Date:                Sun, 06 Dec 2020   Deviance:                       436.86
Time:                        18:18:04   Pearson chi2:                     381.
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.4397      1.120     

Day of week, Month, Saury, Blue runner, Mackerel, Herring, Bonito, Humbolt squid, summer, and fall were all significant predictors for number of fish eaten for Gray Reefs

Dropping:

In [90]:
drops_concatGR = drops_concat[drops_concat['All_GR'].notna()]

In [91]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concatGR)) < 0.8
drops_train = drops_concatGR[mask]
drops_test = drops_concatGR[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=367
Testing data set length=71


In [92]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [93]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_GR   No. Observations:                  244
Model:                            GLM   Df Residuals:                      217
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -317.50
Date:                Sun, 06 Dec 2020   Deviance:                       317.41
Time:                        18:18:14   Pearson chi2:                     310.
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          16.3775      6.412     

Month, Temperature, Blue Runner, and light training were significant predictors for number of drops for Gray Reefs

Targeting:

In [94]:
targets_concatGR = targets_concat[targets_concat['All_GR'].notna()]

In [95]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concatGR)) < 0.8
targets_train = targets_concatGR[mask]
targets_test = targets_concatGR[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=350
Testing data set length=87


In [96]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [97]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_GR   No. Observations:                  230
Model:                            GLM   Df Residuals:                      203
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -916.38
Date:                Sun, 06 Dec 2020   Deviance:                       834.92
Time:                        18:18:22   Pearson chi2:                     813.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -4.1625      1.478     

Month, Temperature, Saury, Blue runner, Mackerel, Herring, Sardine, Mazuri vitamins, Salmon, Bluefish, Mahi, Humbolt squid, GR location, covid, fall, and group feed were all significant predictors for the number of targets for Gray Reefs

# Poisson Regression for Each Gender of Shark

# Male

Eating:

In [98]:
expr = """male ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer +
fall + spring + GroupFeed"""

In [99]:
eat_concatM = eat_concat[eat_concat['male'].notna()]

In [100]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concatM)) < 0.8
eat_train = eat_concatM[mask]
eat_test = eat_concatM[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=467
Testing data set length=115


In [101]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [102]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   male   No. Observations:                  235
Model:                            GLM   Df Residuals:                      208
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -660.35
Date:                Sun, 06 Dec 2020   Deviance:                       312.93
Time:                        18:18:49   Pearson chi2:                     304.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           3.6099      1.774     

Herring, Mazuri Vitamins, Salmon, Bluefish, BT_SB Location, and light training were all significant predictors for the number of fish eaten for male sharks

Dropping:

In [103]:
drops_concatM = drops_concat[drops_concat['male'].notna()]

In [104]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concatM)) < 0.8
drops_train = drops_concatM[mask]
drops_test = drops_concatM[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=482
Testing data set length=100


In [105]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [106]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   male   No. Observations:                  243
Model:                            GLM   Df Residuals:                      216
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -430.87
Date:                Sun, 06 Dec 2020   Deviance:                       361.61
Time:                        18:18:56   Pearson chi2:                     329.
No. Iterations:                    20                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         -18.8662   1.36e+04     

Light training and covid were the only significant predictors for number of pieces dropped for male sharks

Targetting:

In [107]:
targets_concatM = targets_concat[targets_concat['male'].notna()]

In [108]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concatM)) < 0.8
targets_train = targets_concatM[mask]
targets_test = targets_concatM[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=487
Testing data set length=95


In [109]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [110]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   male   No. Observations:                  240
Model:                            GLM   Df Residuals:                      213
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1110.4
Date:                Sun, 06 Dec 2020   Deviance:                       952.25
Time:                        18:19:06   Pearson chi2:                     942.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.5359      1.031     

Month, Temperature, Mackerel, Salmon, Bonito, Bluefish, Goggle eye, BT_SB Location, covid, light training, spring, and group feeding were all significant predictors for the number of targets for male sharks

# Female

Eating:

In [111]:
expr = """female ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer +
fall + spring + GroupFeed"""

In [112]:
eat_concatF = eat_concat[eat_concat['female'].notna()]

In [113]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concatF)) < 0.8
eat_train = eat_concatF[mask]
eat_test = eat_concatF[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=470
Testing data set length=112


In [114]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [115]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 female   No. Observations:                  241
Model:                            GLM   Df Residuals:                      214
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -729.61
Date:                Sun, 06 Dec 2020   Deviance:                       403.20
Time:                        18:19:36   Pearson chi2:                     380.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.3368      1.580     

Temperature, Saury, Blue runner, humbolt squid, BT_SB Location, light training, and fall were all significant predictors for number of pieces eaten for female sharks

Dropping:

In [116]:
drops_concatF = drops_concat[drops_concat['female'].notna()]

In [117]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concatF)) < 0.8
drops_train = drops_concatF[mask]
drops_test = drops_concatF[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=464
Testing data set length=118


In [118]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [119]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 female   No. Observations:                  231
Model:                            GLM   Df Residuals:                      204
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -468.52
Date:                Sun, 06 Dec 2020   Deviance:                       348.59
Time:                        18:19:42   Pearson chi2:                     352.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           8.4719      3.828     

Saury, Mackerel, Bluefish, and covid were all significant predictors for the number of pieces dropped for female sharks

Targetting:

In [120]:
targets_concatF = targets_concat[targets_concat['female'].notna()]

In [121]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concatF)) < 0.8
targets_train = targets_concatF[mask]
targets_test = targets_concatF[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=481
Testing data set length=101


In [122]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [123]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 female   No. Observations:                  241
Model:                            GLM   Df Residuals:                      214
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1026.5
Date:                Sun, 06 Dec 2020   Deviance:                       780.57
Time:                        18:19:51   Pearson chi2:                     784.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.1955      1.065     

Day of week, Month, Temperature, Blue runner, Squid, Mackerel, Herring, Mazuri Vitamins, Salmon, Bluefish, Humbolt Squid, BT_SB location, GR location, light training, fall, and spring were all significant predictors for the number of targets for female sharks